## to perform inference using the A3T-GCN model run the cells below and follow the instuctions

In [14]:
import numpy as np
from torch_geometric_temporal.dataset import METRLADatasetLoader
from torch_geometric_temporal.signal import StaticGraphTemporalSignal
loader = METRLADatasetLoader()
dataset = loader.get_dataset(num_timesteps_in=12, num_timesteps_out=12)
import torch
import torch_geometric as tg
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import Node2Vec
from torch.utils.data import DataLoader
from torch_geometric_temporal import temporal_signal_split
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import utils
import torch.nn as nn
import torch.nn.functional as F
from typing import Union, Callable, Optional
import torch.optim as optim
import time
import torch.nn.functional as F
from torch_geometric_temporal.nn.recurrent import A3TGCN
import a3tgcn_model

Data was normalized via Z-Score method with mean = [53.59967    0.4982691] and std = [20.209862    0.28815305]


In [4]:
train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.8)
device = torch.device('cuda') # cuda

# Create model and optimizers
model = a3tgcn_model.TemporalGNN(node_features=2, periods=12).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)


In [6]:
model.load_state_dict(torch.load('final_model_a3tgcn.pt'))

<All keys matched successfully>

In [11]:
def evaluate(model,sensor=100,timestep=1,height=10,width=8):
    model.eval()
    loss = 0
    step = 0
    loss_mae = 0
    horizon = 1000
    mean = 53.59
    std = 20.209
    # Store for analysis
    predictions = []
    labels = []

    for snapshot in test_dataset[2350:3000]:
        x = snapshot.x.to(device)
        #snapshot = snapshot.to(device)
        edge_indexes = snapshot.edge_index.to(device)
        y = snapshot.y.to(device)
        # Get predictions
        y_hat = model(x, edge_indexes)
        # Mean squared error
        loss = loss + torch.mean((y_hat-y)**2)
        loss_mae += torch.mean(torch.abs(y_hat-y))
        # Store for analysis below
        labels.append(y)
        predictions.append(y_hat)
        step += 1
        if step > horizon:
              break

    loss = loss / (step+1)
    loss_mae /= (step+1)
    loss = loss.item()
    loss_mae = loss_mae.item()
    preds = np.asarray([pred[sensor][timestep].detach().cpu().numpy() for pred in predictions])
    labs  = np.asarray([label[sensor][timestep].cpu().numpy() for label in labels])
    unnorm_preds = preds*std+mean
    unnorm_labs = labs*20.209+53.59
    plt.figure(figsize=(height,width))
    plt.plot(unnorm_labs, label="true")
    plt.plot(unnorm_preds, label="pred")
    plt.xlabel('Timestep')
    plt.ylabel('Speed')
    plt.legend()
    plt.savefig('sensor_eg1.png')
    plt.show()

### run the cell below with arguments passed for sensor number [0,206], timestep [0,11] , image height and image width. 

In [ ]:
'''
Evaluate function plots predicted speed values against the true speed values using the trained A3TGCN model.
'''
evaluate(model,sensor=150,timestep=1,height=10,width=8)